In [12]:
#import tensorflow as tf
import datetime
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import TensorBoard
import os, shutil
from PIL import Image, ImageDraw, ImageFont, ImageColor
import random
import pandas as pd

In [2]:
%load_ext tensorboard

In [13]:
train_dataset_dir ='\\Users\\danie\\Penn_State_REU_Jupyter\\Hexagon ML Project GitHub\\Training_set_6_15' 
test_dataset_dir='\\Users\\danie\\Penn_State_REU_Jupyter\\Hexagon ML Project GitHub\\Testing_set_6_16' 

df=pd.read_csv(os.path.join(train_dataset_dir,"training_dataframe.csv"))
df = df.sample(frac=1,random_state=5).reset_index(drop=True)

dft = pd.read_csv(os.path.join(test_dataset_dir,"training_dataframe.csv"))
dft = dft.sample(frac=1,random_state=5).reset_index(drop=True)

In [17]:
df.shape
dft.shape

(10935, 9)

In [5]:
def build_model():
    model = models.Sequential()                                  
    model.add(layers.Conv2D(16, (3,3) ,activation='relu', input_shape=(32,32,3)))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Conv2D(16,(3,3), activation='relu'))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Flatten())
    #model.add(layers.Dropout(0.2))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(8))
    return model

In [18]:
#Creates generators for training and validation images
SEED=5
validation_split_frac=0.2
batch_size=256
train_datagen = ImageDataGenerator(rescale=1./255,validation_split=validation_split_frac)
test_datagen = ImageDataGenerator(rescale=1./255)  
train_generator= train_datagen.flow_from_dataframe(
        df,
        directory=train_dataset_dir,
        x_col="filename",
        y_col=['x_coord','y_coord','neighbor1x','neighbor1y','neighbor2x','neighbor2y','neighbor3x','neighbor3y'],
        target_size=(32,32),
        shuffle=True,
        seed= SEED,
        class_mode="raw",
        batch_size=batch_size,
        subset='training')

validation_generator= train_datagen.flow_from_dataframe(
        df,
        directory=train_dataset_dir,
        x_col="filename",
        y_col=['x_coord','y_coord','neighbor1x','neighbor1y','neighbor2x','neighbor2y','neighbor3x','neighbor3y'],
        target_size=(32,32),
        shuffle=True,
        seed= SEED,
        class_mode="raw",
        batch_size=batch_size,
        subset='validation')

test_generator= test_datagen.flow_from_dataframe(
        dft,
        directory=test_dataset_dir,
        x_col="filename",
        y_col=['x_coord','y_coord','neighbor1x','neighbor1y','neighbor2x','neighbor2y','neighbor3x','neighbor3y'],
        target_size=(32,32),
        shuffle=True,
        seed= SEED,
        class_mode="raw",
        batch_size=batch_size)

Found 97200 validated image filenames.
Found 24300 validated image filenames.
Found 10935 validated image filenames.


In [9]:
# from tensorboard import notebook
# notebook.list()
%tensorboard --logdir Training_logs/

In [10]:
model=build_model()
model.compile(optimizer='sgd', loss='mse', metrics=['mae'])
#model.summary()
num_epochs=60
training_steps=int(df.shape[0]*(1-validation_split_frac)/batch_size)
validation_steps=int(df.shape[0]*validation_split_frac/batch_size)
#assert (training_steps+validation_steps)*batch_size == df.shape[0]
log_dir = os.path.join("Training_logs\\fit", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
model.fit(
    train_generator,
    steps_per_epoch=training_steps,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    epochs=num_epochs,
    verbose=1, 
    callbacks=[tensorboard_callback]
    )


  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 379 steps, validate for 94 steps
Epoch 1/60
379/379 [==============================] - 54s 142ms/step - loss: 0.0327 - mae: 0.1529 - val_loss: 0.0270 - val_mae: 0.1402
Epoch 2/60
379/379 [==============================] - 48s 128ms/step - loss: 0.0265 - mae: 0.1388 - val_loss: 0.0262 - val_mae: 0.1379
Epoch 3/60
379/379 [==============================] - 49s 129ms/step - loss: 0.0259 - mae: 0.1369 - val_loss: 0.0256 - val_mae: 0.1360
Epoch 4/60
379/379 [==============================] - 48s 126ms/step - loss: 0.0252 - mae: 0.1350 - val_loss: 0.0248 - val_mae: 0.1340
Epoch 5/60
379/379 [==============================] - 48s 125ms/step - loss: 0.0245 - mae: 0.1327 - val_loss: 0.0240 - val_mae: 0.1315
Epoch 6/60
379/379 [==============================] - 47s 125ms/step - loss: 0.0236 - mae: 0.1300 - val_loss: 0.0231 - val_mae: 0.1285
Epoch 7/60
379/379 [==============================] - 47s 125ms/step - loss: 0.0226 - mae: 0.1268

In [11]:
model.save("60epochrun6_16")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: 60epochrun6_16\assets


In [19]:
model=models.load_model('60epochrun6_16')

In [22]:
#Evaluates the model on testing set
testing_steps=dft.shape[0]//batch_size
log_dir = os.path.join("Training_logs\\test", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
model.evaluate(
    test_generator,
    steps=testing_steps
    #callbacks=[tensorboard_callback]
)

  ...
    to  
  ['...']
42/42 [==============================] - 4s 91ms/step - loss: 0.0112 - mae: 0.0784


[0.011231104200262399, 0.078366056]

['loss', 'mae']